# BERTopic model

In [1]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN

import tensorflow as tf
# from cuml.cluster import HDBSCAN
# from cuml.manifold import UMAP
# from cuml.preprocessing import normalize

from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer 
import string
import re
import nltk
from bs4 import BeautifulSoup   
import contractions,unicodedata


nltk.download('stopwords')                              # Download Stopwords.
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import PorterStemmer                     # Stemmer
from nltk.corpus import stopwords                       # Import stopwords.
from nltk.tokenize import word_tokenize, sent_tokenize  # Import Tokenizer.
from nltk.stem.wordnet import WordNetLemmatizer         # Import Lemmatizer.


import spacy
from sklearn.cluster import KMeans
import plotly.express as px

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\p_uli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\p_uli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\p_uli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
data= pd.read_csv(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\data_clean_processed_sample.csv')

In [3]:
data.head()

,email_id,subject,email,bow,embeddings
0,saibi-e\inbox\612#1,WSCC (Mahave 02) (San Juan 03),"Dear Power Outage Database Customer,\n\nAttach...",dear power outage database customer attached f...,"dear power outage database customer, attached ..."
1,kitchen-l\_americas\netco_restart\28#2,RE:,Ryan is waffling. I talked to him this weeken...,ryan waffling talked weekend said wife want mo...,ryan waffling. talked weekend said wife want m...
2,motley-m\sent_items\1#1,SSARR Update Available Now,"<center><LI><a Href=""http://www.nwrfc.noaa.gov...",columbia river basin lower snake updated mar,columbia river basin lower snake.....updated: mar
3,kaminski-v\all_documents\577#1,Credit Risk Model Comments - at this point.,"Hi everyone,\n\nI have run the model and, alon...",everyone run model along contract brief questi...,"everyone, run model and, along contract brief ..."
4,NaN,Confirmation: Risk Management Simulation Mee...,"Hi Dutch, This message is to confirm ou...",dutch message confirm meeting thursday novembe...,"dutch, message confirm meeting on, thursday, n..."


## Data processing

In [4]:
data['email'] = data['email'].str.strip()
#data['subject'] = data['subject'].str.strip()

In [5]:
# utl removal
def remove_url(text):
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    #text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(regex, '', text, flags=re.MULTILINE)
    return text
data['clean_text'] = data['email'].apply(lambda x: remove_url(x))

In [6]:
# html tag removal
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

data['clean_text'] = data['clean_text'].apply(lambda x: strip_html(x))

In [7]:
# contractions removal
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

data['clean_text'] = data['clean_text'].apply(lambda x: replace_contractions(x))
data.head()

,Unnamed: 0,email_id,email,subject,clean_text
0,0,allen-p\all_documents\100#2,"PERSON , \r\n\r\n Please use the second check ...",Consolidated positions : Issues & To Do list,"PERSON , \r\n\r\n Please use the second check ..."
1,1,allen-p\all_documents\101#2,STRUCTURE : \r\n Typically the structure is a ...,Consolidated positions : Issues & To Do list,STRUCTURE : \r\n Typically the structure is a ...
2,3,allen-p\all_documents\103#1,---------------------- Forwarded by PERSON on ...,Re : 2001 Margin Plan,---------------------- Forwarded by PERSON on ...
3,4,allen-p\all_documents\104#1,Internet Data Gain Is a Major Power Drain on \...,"Var, Reporting and Resources Meeting",Internet Data Gain Is a Major Power Drain on \...
4,5,allen-p\all_documents\105#1,Do not underestimate the effects of the Intern...,Westgate,Do not underestimate the effects of the Intern...


In [8]:
# Removing numbers
def remove_numbers(text):
    text = re.sub(r'\d+', '', text)
    return text

data['clean_text'] = data['clean_text'].apply(lambda x: remove_numbers(x))

In [9]:
# tokenization
data['clean_text'] = data.apply(lambda row: nltk.word_tokenize(row['clean_text']), axis=1) 

In [10]:
%%time
lemmatizer = WordNetLemmatizer()
stopwords = stopwords.words('english')

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords:
            new_words.append(word)
    return new_words

def lemmatize_list(words):
    new_words = []
    for word in words:
        new_words.append(lemmatizer.lemmatize(word, pos='v'))
    return new_words

# def mask_names(words):
#     new_words = []
#     for word in words:
#         doc=nlp(word)
#         if doc[0].ent_type_=='PERSON':
#             new_words.append('PERSON')
#         else:
#             new_words.append(word)
#     return new_words

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    words = lemmatize_list(words)
#     words = mask_names(words)
    return ' '.join(words)

data['clean_text'] = data.apply(lambda row: normalize(row['clean_text']), axis=1)
data.head()

CPU times: total: 19.4 s
Wall time: 32.9 s


,Unnamed: 0,email_id,email,subject,clean_text
0,0,allen-p\all_documents\100#2,"PERSON , \r\n\r\n Please use the second check ...",Consolidated positions : Issues & To Do list,person please use second check october payment...
1,1,allen-p\all_documents\101#2,STRUCTURE : \r\n Typically the structure is a ...,Consolidated positions : Issues & To Do list,structure typically structure limit partnershi...
2,3,allen-p\all_documents\103#1,---------------------- Forwarded by PERSON on ...,Re : 2001 Margin Plan,forward person pm invitation chairperson perso...
3,4,allen-p\all_documents\104#1,Internet Data Gain Is a Major Power Drain on \...,"Var, Reporting and Resources Meeting",internet data gain major power drain local uti...
4,5,allen-p\all_documents\105#1,Do not underestimate the effects of the Intern...,Westgate,underestimate effect internet economy load gro...


In [213]:
# drop NAs
data.dropna(subset=['clean_text'], inplace=True)

In [11]:
nlp = spacy.load('en_core_web_trf')
nlp.add_pipe("merge_entities")

def get_ner(text):
    doc = nlp(text)
    return " ".join(['PERSON' if t.ent_type_ and t.ent_type_ == 'PERSON' else t.text for t in doc])


In [214]:
#data.to_csv(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\mask_name_cleaned.csv',index_label=False)

In [2]:
data= pd.read_csv(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\data_clean_processed_sample.csv')
data.head()

,email_id,subject,email,bow,embeddings
0,saibi-e\inbox\612#1,WSCC (Mahave 02) (San Juan 03),"Dear Power Outage Database Customer,\n\nAttach...",dear power outage database customer attached f...,"dear power outage database customer, attached ..."
1,kitchen-l\_americas\netco_restart\28#2,RE:,Ryan is waffling. I talked to him this weeken...,ryan waffling talked weekend said wife want mo...,ryan waffling. talked weekend said wife want m...
2,motley-m\sent_items\1#1,SSARR Update Available Now,"<center><LI><a Href=""http://www.nwrfc.noaa.gov...",columbia river basin lower snake updated mar,columbia river basin lower snake.....updated: mar
3,kaminski-v\all_documents\577#1,Credit Risk Model Comments - at this point.,"Hi everyone,\n\nI have run the model and, alon...",everyone run model along contract brief questi...,"everyone, run model and, along contract brief ..."
4,quigley-d\sent_items\480#1,Confirmation: Risk Management Simulation Mee...,"Hi Dutch, This message is to confirm ou...",dutch message confirm meeting thursday novembe...,"dutch, message confirm meeting on, thursday, n..."


In [3]:
#data2= data.sample(n=20000)
if type(data['embeddings']) is list:
    text = data['embeddings']
else:
    text = data['embeddings'].tolist()

text=[str(x) for x in text]

In [4]:
# vocab= pd.read_csv(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\clean_vocab.csv')
# vocab.head()

## Model

In [5]:
# Defining the grid to implement Random Search CV

## BERTopic hyperparameters
# transformers. Taken from https://www.sbert.net/docs/pretrained_models.html
s_transf=['all-MiniLM-L6-v2','all-MiniLM-L12-v2']

# top n words per topic, it is recommended to keep this number between 10-20
top_n_words=[int(x) for x in np.linspace(10,20,6)]

# ngram range: Let's try 1,2 and 3-grams
ngram_r=[(1,1),(1,2),(1,3)]

# min topic size
min_topic_s=[int(x) for x in np.linspace(10,200,6)]

## UMAP hyperparameters
# number of neighbors
n_neigh=[int(x) for x in np.linspace(10,50,6)]

## HDBSCAN hyperparameters
# min cluster size
min_cl_s=[int(x) for x in np.linspace(50,250,6)]



In [41]:
# %%time
# with tf.device('/GPU:0'):
#     # embedding
#     sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
#     #sentence_model = SentenceTransformer('all-MiniLM-L12-v2')
#     embeddings = sentence_model.encode(text, show_progress_bar=True)

# all_embeddings = np.array(embeddings)
# np.save('embeddings_L6-v2.npy', all_embeddings)
embeddings= np.load('embeddings_L6-v2.npy')
#embeddings= np.load('embeddings.npy')

In [42]:
def create_model(topwords,n_gram,min_top, nn, min_cluster):
    from nltk.corpus import stopwords
    stop_w = stopwords.words('english')
    stop_w = stop_w+['imagemasker','urlmasker','tofrom','namemasker','im','smith','john','pm']
    
    # ngrams
    vectorizer_model = CountVectorizer(ngram_range=n_gram, stop_words=stop_w)
    
    # UMAP: dimensionality reduction
    umap_model = UMAP(n_neighbors=nn, min_dist=0.0, metric='cosine', random_state=42)
    
    # HDBSCAN: clustering
    hdbscan_model = HDBSCAN(min_cluster_size=min_cluster, metric='euclidean', 
                            cluster_selection_method='eom', prediction_data=True, min_samples=5)
    
    cluster_model = KMeans(n_clusters=min_top)
    
    # BERTopic
    model = BERTopic(
        top_n_words=topwords,
        #min_topic_size=min_top,
        nr_topics=min_top,
        vectorizer_model=vectorizer_model,
        language='english', calculate_probabilities=True,
        #embedding_model=sentence_model,
        verbose=True,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
    )
    
    return model

In [43]:
model=create_model(10,ngram_r[1],min_topic_s[0],10,min_cl_s[0])
model

In [44]:
%%time
with tf.device('/GPU:0'):
    topics, probs = model.fit_transform(text,embeddings)


2022-10-31 11:38:26,546 - BERTopic - Reduced dimensionality
2022-10-31 11:38:34,985 - BERTopic - Clustered reduced embeddings
2022-10-31 11:38:48,381 - BERTopic - Reduced number of topics from 106 to 11


CPU times: total: 42.5 s
Wall time: 31.9 s


In [46]:
# save the model
aux=[s_transf[1],10,ngram_r[1],min_topic_s[0],10,min_cl_s[0]]
aux=[str(x) for x in aux]
model.save("_".join(aux))
#model=BERTopic.load("_".join(aux))


C:\Users\p_uli\anaconda3\envs\tf-gpu\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [64]:
probs=model.probabilities_
topics=model.topics_

## Results

In [65]:
freq = model.get_topic_info()
freq.head(11)

,Topic,Count,Name
0,-1,15742,-1_please_enron_would_new
1,0,581,0_think_know_let_funny
2,1,527,1_deal_price_volume_day
3,2,512,2_power_energy_california_said
4,3,473,3_email_please_copy_fax
5,4,460,4_agreement_please_attached_draft
6,5,386,5_going_get_night_good
7,6,347,6_recipient_intended_intended recipient_enron
8,7,341,7_thanks_call_thank_cell
9,8,321,8_meeting_call_conference_please


In [66]:
# create topic assignment per email
topics=[]
probas=[]
for i in range(data.shape[0]):
    topics.append(np.argmax(probs[i]))
    probas.append(np.max(probs[i]))


In [67]:
# create the table with the 10 most important words per topic
words=[]
c_tfidf=[]
top=[]
for i in range(0,10):
    top= top+ [i]*10
    for x in model.get_topic(i): #c-TF-IDF scores
        words.append(x[0])
        c_tfidf.append(x[1])
    


In [68]:
# 
data['topic']=topics
data['probs']= probas

In [69]:
# topic words
top_words_topic=pd.DataFrame(list(zip(top,words,c_tfidf)),columns=['topic','top_words','c-tf-idf'])

In [70]:
top_words_topic.head(20)

,topic,top_words,c-tf-idf
0,0,think,0.066124
1,0,know,0.056768
2,0,let,0.040311
3,0,funny,0.035061
4,0,let know,0.031001
5,0,fine,0.030508
6,0,see,0.030505
7,0,please,0.028863
8,0,thanks,0.028621
9,0,one,0.026303


In [71]:
#file_name=[s_transf[0],top_n_words[0],ngram_r[0],min_topic_s[0],10,min_cl_s[0]]
file_name=[s_transf[0],10,ngram_r[1],min_topic_s[0],10,min_cl_s[0]]
file_name=[str(x) for x in file_name]
file_name="_".join(file_name)


In [72]:
top_words_topic.to_csv(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\BERTopic\top_words_{}.csv'.format(file_name),index=False)
data.to_csv(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\BERTopic\assignments_{}.csv'.format(file_name),index=False)

## Visualization

In [40]:
model.visualize_barchart(top_n_topics=7)

In [95]:
model.visualize_barchart(top_n_topics=10)

In [115]:
fig=model.visualize_topics()
fig.write_html(r"C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Images\BERTopic\Distance_{}.html".format(file_name))

In [116]:
hierarchy=model.visualize_hierarchy()
hierarchy.write_html(r"C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Images\BERTopic\Hierarchy_{}.html".format(file_name))

In [117]:
docs_viz=model.visualize_documents(text,embeddings=embeddings)
docs_viz.write_html(r"C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Images\BERTopic\Docs_{}.html".format(file_name))

In [118]:
bars=model.visualize_barchart(top_n_topics=10)
bars.write_html(r"C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Images\BERTopic\Bars_{}.html".format(file_name))


In [147]:
# # Document visualization (left fot the best models)

# from umap import UMAP
# # Prepare embeddings

# sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
# embeddings = sentence_model.encode(text, show_progress_bar=False)

# # Train BERTopic
# topic_model = BERTopic(nr_topics=10,).fit(text, embeddings)

# # Run the visualization with the original embeddings
# topic_model.visualize_documents(text, embeddings=embeddings)

# # Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
# reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
# topic_model.visualize_documents(text, reduced_embeddings=reduced_embeddings)


## ejemplos de casos raros

In [70]:
# examples from fyi fyi
indices=[]
for i in range(data.shape[0]):
    email=data.iloc[i]['clean_text']
    if email.find('fyi')>0:
        indices.append(i)
        

CountVectorizer(ngram_range=(1, 2), stop_words='english')

In [100]:
X2 = model.vectorizer_model.transform(text)
aux=model.vectorizer_model.get_feature_names()

In [102]:
model.vectorizer_model.vocabulary_['fyi fyi']

407629

In [80]:
for i in indices:
    email=data.iloc[i]['clean_text']
    if len(email.split())<5:
        print(email)


thank
thank
thank
homework
consider do thank person
hat
latest comment fyi ngdoc
enjoy
info
thank
apologize mean sorry confusion
update tomorrow
fyi
fyi
fyi
fyi
keep post thank bob
fyi
fyi yes response lengthy
fyi
fyi
fyi
person fyi slide helpful
fyi mm
fyi
fyi patti
fyi mm
thank head
fyi
fyi
per person
fyi
fyi jr jr
fyi
fyi
fyi
fyi
fyi
phone number ava
fyi
fyi britt
lynn think
fyi
fyi
fyi
fyi thank lynn
fyi thank lynn
fyi thank lynn
person anything thank lynn
john fyi thank lynn
person thank lynn
fyi thank lynn
fyi thank lynn
sound good thank lynn
fyi thank lynn
fyi thank lynn
mine contact person
fyi
fyi
thank convince person
fyi
fyi
fyi
fyi
help person
fyi
fyi
data
fyi
fyi
cathy answer cirino
voice mail
thank much
fyi
fyi
give person thank
help
fyi lizzette
great result
fyi michelle
thank get info person
thank
get thank monika
ok try thank monika
thank finally open
thank
person see comment
fyi thank lynn
fyi
thank update
help
thank person excellent plan
thank keep post
fyi
fyi piece


In [59]:
data.iloc[13775]['clean_text']

'vasant fyi'

In [61]:
for i in indices:
    email=data.iloc[i]['clean_text']
    if len(email.split())<3:
        print(email)
    

shipper fyi
daren fyi
daren fyi
daren fyi
vasant fyi
person fyi
presly fyi
calendar fyi
ken fyi
kay fyi
person fyi
person fyi
